# 1. Import Dependencies

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

## Dataset
This specific dataset seperates hand gestures into 4 different classes.

- stop
- onefingerup
- twofingerup
- thumbsup

The information about each hand gesture is contained into 42 features.

In [2]:
CSV_COLUMN_NAMES = np.arange(43)
CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]
CSV_COLUMN_NAMES[42] = "Species"
SPECIES = ['stop', 'onefingerup', 'twofingerup', 'thumbsup']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_0.csv")

In [3]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [4]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,Species
140,5,49,-13,46,-24,38,-25,40,-26,46,...,42,40,11,30,23,20,42,12,54,2
109,13,108,-28,108,-59,80,-38,55,-1,48,...,68,71,24,59,38,39,71,25,91,2
162,46,12,36,-13,18,-37,8,-57,2,-74,...,17,13,32,-6,33,-2,32,3,31,3
22,0,151,-44,132,-74,94,-90,55,-108,23,...,-90,58,50,71,5,75,-22,76,-47,0
39,27,104,-13,102,-42,79,-60,53,-79,33,...,-69,53,28,56,-4,56,-26,56,-44,0


In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
140,5,49,-13,46,-24,38,-25,40,-26,46,...,1,42,40,11,30,23,20,42,12,54
109,13,108,-28,108,-59,80,-38,55,-1,48,...,8,68,71,24,59,38,39,71,25,91
162,46,12,36,-13,18,-37,8,-57,2,-74,...,0,17,13,32,-6,33,-2,32,3,31
22,0,151,-44,132,-74,94,-90,55,-108,23,...,35,-90,58,50,71,5,75,-22,76,-47
39,27,104,-13,102,-42,79,-60,53,-79,33,...,13,-69,53,28,56,-4,56,-26,56,-44


In [6]:
train.shape  # we have 120 entires with 4 features

(159, 42)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [7]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [8]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [9]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model")

In [10]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 4 classes.
    n_classes=len(SPECIES),
    model_dir= MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Tensorflow\\workspace\\models\\my_hand_gesture_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [11]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 36.114677, step = 0
INFO:tensorflow:global_step/sec: 81.9625
INFO:tensorflow:loss = 1.8021538, step = 100 (1.232 sec)
INFO:tensorflow:global_step/sec: 98.8085
INFO:tensorflow:loss = 0.9610002, step = 200 (1.002 sec

## Export Model

In [12]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Tensorflow\workspace\models\my_hand_gesture_model\temp-1636548219\saved_model.pbtxt


## Load Model

In [13]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

<ConcreteFunction pruned(examples) at 0x3CB14BB0>

In [14]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert from list to tensor
examples = tf.constant(examples)

print(test)
print(feature)

      0    1   2    3   4    5    6    7    8    9  ...  32   33  34  35  36  \
1    35  153 -19  126 -56   90  -79   55  -97   27  ...  24  -83  60  36  65   
7    35  150 -16  138 -55  103  -79   66 -103   36  ...  29  -95  64  39  70   
14   22  101 -12   93 -38   68  -53   44  -67   25  ...  17  -64  49  29  52   
16   19   84  -9   83 -31   63  -42   44  -54   32  ...  18  -61  39  29  46   
20   30  100  -6   95 -36   74  -56   52  -76   35  ...  12  -64  52  23  53   
23   30  149 -16  125 -51   90  -71   54  -91   24  ...  22  -77  60  38  64   
26   43  207 -20  197 -74  152 -109  100 -134   50  ...  46 -131  71  54  81   
35   32  123 -13  119 -46   90  -66   58  -88   34  ...  15  -78  62  32  64   
42   32  176 -24  157 -59  109  -71   60  -91   22  ...  21 -101  67  51  66   
51   24   56  -2   59 -36   43  -49   19  -31    2  ...  15   37  54  -8  37   
54   18   58 -10   60 -43   40  -51   20  -31   10  ...  17   37  51  -9  45   
55   19   59  -8   61 -41   40  -48   20

In [15]:
# make predictions of all testset
predictions = predict_fn(examples=examples)
print(predictions)

{'classes': <tf.Tensor: shape=(40, 1), dtype=string, numpy=
array([[b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'0'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'1'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'2'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3'],
       [b'3']], dtype=object)>, 'all_class_ids': <tf.Tensor: shape=(40, 4), dtype=int32, numpy=
array([[0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       [0, 1, 2, 3],
       

In [16]:
# print results
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {SPECIES[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {SPECIES[test_y.iloc[idx]]}")

tf.Tensor(0.9999672, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(0.9999932, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(0.99968636, shape=(), dtype=float32)
	Prediction is stop 99.97%
	Expected: stop
tf.Tensor(0.99949026, shape=(), dtype=float32)
	Prediction is stop 99.95%
	Expected: stop
tf.Tensor(0.9994684, shape=(), dtype=float32)
	Prediction is stop 99.95%
	Expected: stop
tf.Tensor(0.99991214, shape=(), dtype=float32)
	Prediction is stop 99.99%
	Expected: stop
tf.Tensor(0.9999999, shape=(), dtype=float32)
	Prediction is stop 100.00%
	Expected: stop
tf.Tensor(0.99992585, shape=(), dtype=float32)
	Prediction is stop 99.99%
	Expected: stop
tf.Tensor(0.9992781, shape=(), dtype=float32)
	Prediction is stop 99.93%
	Expected: stop
tf.Tensor(0.9994843, shape=(), dtype=float32)
	Prediction is onefingerup 99.95%
	Expected: onefingerup
tf.Tensor(0.99874794, shape=(), dtype=float32)
	Prediction is onefingerup 99.87%
	Expecte

## Evaluation

In [17]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-10T13:43:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.52403s
INFO:tensorflow:Finished evaluation at 2021-11-10-13:43:44
INFO:tensorflow:Saving dict for global step 5000: accuracy = 1.0, average_loss = 0.018036956, global_step = 5000, loss = 0.018036956
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000

Test set accuracy: 1.000



## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [18]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = SPECIES[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES[class_id], 100 * probability))
        print(f"Expected is {expected}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "stop" (100.0%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "stop" (100.0%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.